# Regression: Boston Housing Dataset

Quelle: https://www.kaggle.com/altavish/boston-housing-dataset  

Der Datensatz enthält Daten über die Lebensbedingungen in den Außenbezirken von Boston im Jahr 1978.  
Er wurde von David Harrison und Daniel L. Rubinfeld veröffentlicht. 

<hr style="border:1px solid gray"> </hr>

## Inhalt

1. [Geschäftverständnis / Aufgabe](#kap1)


2. [Datenverständnis](#kap2)  


3. [Datenvorbereitung](#kap3)  
    3.1 [Aufteilung in Trainings- und Testdatensatz](#kap31)  
    3.2 [Normalisierung der Daten](#kap32)  
    3.3 [Erste Merkmalsauswahl](#kap33)  
    3.4 [Visualisierung](#kap34)  


4. [Modellierung](#kap4)  
    4.1 [Lineare Regression](#kap41)  
    4.2 [Gütemaße für Regressionsfunktionen](#kap42)  
    4.3 [Polynomiale Regression](#kap43)  
    4.4 [Mehrdimensionale Regression](#kap44)  
    4.5 [Lasso](#kap45)  


5. [Fazit](#kap5)

<hr style="border:1px solid gray"> </hr>

## 1. Geschäftsverständnis / Aufgabe <a name="kap1"></a>

Ein Datensatz, der verschiedene Kennwerte zu Wohnungen in Boston enthält, soll genutzt werden, um ein Vorhersagemodell zu entwickeln, das den Wert einer Wohnung bestimmt.

Der Unterschied zum Kapitel Klassifikation besteht darin, dass das vorherzusagende Merkmal ein metrisch skaliertes Merkmal und keine Klasse (Bsp. end-to-end: `Überlebt`/`Nicht überlebt`) ist. Ebenso sind die vorhersagenden Merkmale metrisch skaliert.

##  2. Datenverständnis <a name="kap2"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
csv_path = "M5_Video_BostonHousing.csv"  
df = pd.read_csv(csv_path)  
df

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 

Erforschen Sie den Datensatz, indem Sie die Kommentare zu den Merkmalen im Notebook lesen, den Befehl .info anwenden und ein erstes Histogramm erzeugen. Löschen Sie außerdem alle Beispiele mit NaN Einträge des Datensatzes heraus.

</div>

Der Datensatz enthält 490 Zeilen, dies sind die Vorstadtbezirke von Boston, über die die Daten erhoben wurden. Es wurden jeweils 14 Eigenschaften (=Spalten) erfasst. Die Namen der Spalten bedeuten folgendes:

- `CRIM` gibt die Pro-Kopf-Kriminalitätsrate in dem Vorstadtbezirk an. 
- `ZN` ist der Anteil der Landparzellen, die größer als 25.000 Quadratfuß, etwa 2.320 Quadratmeter, sind.
- Der Flächenanteil der Gebiete, die nicht vom Einzelhandel genutzt werden, ist in `INDUS` erfasst.
- `CHAS` ist die sog. Charles-River-Variable. Sie hat den Wert '1', wenn der Charles River durch den Bezirk fließt und den Wert '0', wenn dies nicht der Fall ist. 
- `NOX` entspricht der Stickstoffmonixid-Konzentration (in Teilen pro 10 Millionen) in diesem Bezirk.
- `RM` gibt die durchschnittliche Anzahl der Zimmer pro Wohnung bzw. Gebäude an.
- `AGE` ist der Anteil der vor 1940 gebauten und von den Eigentümer:innen selbst genutzten Wohnungen. 
- Mit `DIS` wird die gewichteten Entfernunge zu fünf Arbeitsvermittlungsagenturen in Boston angegeben. 
- `RAD` ist ein Index, der die Zugänglichkeit von Einfallsstraßen erfasst.
- `TAX` enthält die Höhe der unverminderten Grundstueuer pro 10.000 Dollar.
- `PTRATIO` gibt das Verhältnis von Schüler:innen zu Lehrer:innen an. 
- `B` errechnet sich nach der Formel $1000 \cdot (\text{a}-0.63)^2$, wobei $\text{a}$ der Anteil der Bevölkerung mit afroamerikanischer Abstammung ist.  
- `LSTAT` ist der Prozentsatz der Bevölkerung mit niedrigem sozialem Status.
- `MEDV` ist der Medianwert des Preises der von den Eigentümern selbst genutzten Wohnungen bzw. Häuser, angegeben in Einheiten von 1.000 Dollar. Hier wird er als durchschnittlicher Wert einer Wohnung in einem Vorstadtbezirk interpretiert. 

Aus dem obigen Datensatz soll der Wert von `MEDV` aus den übrigen Daten vorhergesagt werden. Für unbekannte Daten kann dann eine gute Schätzung abgegeben werden. `MEDV` wird dann als das Zielmerkmal bezeichnet. Die übrigen Spalten sind die Eingabemerkmale. 

Zunächst wird wieder eine allgemeinen Analyse der Daten vorgenommen: 

In [ ]:
df.info()

Alle Werte sind numerisch, sie wurden entweder als float- oder als integer-Werte abgespeichert. In mehreren Spalten fehlen Einträge. Da Datenverständnis und Datenvorbereitung in Modul 3.2. näher erläutert wurden, wird hier der einfache Weg gegangen und die Zeilen mit fehlenden Einträgen gelöscht. 

In [ ]:
df = df.dropna()

In [ ]:
df.hist(figsize=(12,12))
plt.show()

Die Histogramme zeigen, dass es keine Ausreißer gibt und die Merkmale sehr unterschiedlich verteilt sind. Die einzelnen Spalten haben sehr unterschiedliche Wertebereiche. 

## 3. Datenvorbereitung <a name="kap3"></a>

### 3.1 Aufteilung in Trainings- und Testdatensatz <a name="kap31"></a>

Um die Testdaten völlig unabhängig zu halten, wird wieder der  Train-Test-Split vorgenommen:

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, random_state=0, test_size=0.3)

### 3.2 Normalisierung der Daten <a name="kap32"></a>

In diesem Beispiel haben die Daten sehr unterschiedlich große numerische Ausprägungen, wie die nachfolgende Ausgabe zeigt:  

In [ ]:
df.describe()

Die einzelnen Spalten haben sehr unterschiedliche Wertebereiche, z.B. liegen die Werte von `NOX` zwischen 0 und 0.871, aber `ZN` zwischen 0 und 100.  

Normalisierung ist eigentlich erst relevant, wenn Regression in Abhängigkeit von mehreren Merkmalen betrieben wird, wie im zweiten Teil dieses Moduls. Dennoch wird sie hier schon einmal vorgenommen:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_set)                             # Skalierung wird an train_set angepasst
train_set_scaled = scaler.transform(train_set)    # Auf train_set anpassen
test_set_scaled = scaler.transform(test_set)      # Dieselbe Skalierung auch auf test_set anpassen
train_set_scaled = pd.DataFrame(train_set_scaled, index=train_set.index, columns=train_set.columns) # Rückumwandlung in Dataframe
test_set_scaled = pd.DataFrame(test_set_scaled, index=test_set.index, columns=test_set.columns)
train_set_scaled.head()

Die Auswirkung zeigt sich beispielsweise an den beiden nachfolgenden Grafiken: Die Beziehung zwischen den Merkmalen bleibt gleich, lediglich die Skalierung der Achsen unterscheidet sich.

In [ ]:
train_set.plot(kind='scatter', x='LSTAT', y='RM');

In [ ]:
train_set_scaled.plot(kind='scatter', x='LSTAT', y='RM');

### 3.3 Erste Merkmalsauswahl <a name="kap33"></a>

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 

Bestimmen Sie nun durch Berechnung der Korrelationsmatrix, welche Merkmale den größten Einfluss auf das Zielmerkmal `MEDV` haben.
Wählen Sie alle Merkmale, deren Korrelation zu `MEDV` größer als 0,5 ist. 
</div>

In [ ]:
# Platz für Arbeitsauftrag

Die Korrelationsmatrix zeigt, dass die betragsmäßig höchsten Koeffizienten mit `MEDV` die Merkmale `LSTAT`, `RM`, `PTRATIO`, `TAX` und `INDUS` sind.
Diese Merkmale sind aber nicht unabhängig voneinander, z.B. beträgt der Korrelationskoeffizient zwischen `LSTAT` und `RM` -0.67. Beide Merkmale drücken evtl. die gleiche Kausalität aus. 

Zunächst soll ein einfaches Modell erstellt werden:
Als Eingabemerkmal wird `LSTAT` verwendet, weil es den beträgsmäßig größten Korrelationskoeffizienten mit dem Zielmerkmal `MEDV` aufweist.   

### 3.4 Visualisierung <a name="kap34"></a>

Um einen Eindruck der Beziehung zwischen `LSTAT` und `MEDV` zu bekommen, wird das zugehörige Streudiagramm geplottet: 

In [ ]:
train_set.plot(kind='scatter', x='LSTAT', y='MEDV');

Eine Gerade, die von links oben nach rechts unten verläuft, würde den Trend bzw. den groben Zusammenhang zwischen `LSTAT` und `MEDV` schon gut charakterisieren. Welche Gerade dies am besten tut, wird nun berechnet. 

## 4. Modellierung <a name="kap4"></a>

Zunächst werden Trainings- und Testdaten auf die gewünschten Merkmale eingeschränkt: 

In [ ]:
X_train = train_set_scaled[['LSTAT']]
y_train = train_set_scaled[['MEDV']]
X_test = test_set_scaled[['LSTAT']]
y_test = test_set_scaled[['MEDV']]

### 4.1 Lineare Regression <a name="kap41"></a>

Nun wird das lineare Regressionmodell erstellt. Die Syntax entspricht dabei dem bereits bekannten Vorgehen. 

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print('Achsenabschnitt = ', lin_reg.intercept_, ',   Steigung = ', lin_reg.coef_)

Es beschreibt die Gerade $ \text{MEDV} = -0.71963166 \cdot \text{LSTAT} +  0.60635174 $ und bezieht sich auf die normierten Werte. 

In [ ]:
xline = np.linspace(-0.1,1.1, 100).reshape(100, 1)  
yline = lin_reg.predict(xline)

plt.plot(X_train, y_train,'.')
plt.plot(xline,yline , color='red')
plt.xlabel('LSTAT')
plt.ylabel('MEDV')
plt.xlim(-0.1, 1.1)
plt.show()

Die Gerade gibt den Trend der Punktwolke gut wider. Es ist aber auch deutlich zu erkennen, dass eine gebogene Linie den Trend besser annähern könnte. Mehr hierzu folgt in Kapitel 4.3. 

### 4.2 Gütemaße für Regressionsfunktionen <a name="kap42"></a>

Um verschiedene Regressionsfunktionen miteinander vergleichen zu können, wird ein Gütemaß benötigt. Dieses wurde schon bei der Herleitung der Regressionsfunktion benutzt:
Die mittlere quadratische Abweichung = mse (für mean squarred error). 
Es ist brauchbar für den Vergleich von Regressionsfunktionen zu **demselben Datensatz**. 

Allgemeiner ist das Bestimmtheitsmaß R². Der Wertebereich des Bestimmtheitsmaßes liegt grundsätzlich zwischen 0 und 1, bei sehr schlechter Anpassung kann es auch negativ werden. Je näher sein Wert an 1 liegt, desto besser ist die Regressionsfunktion an die Daten angepasst. 

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred_train = lin_reg.predict(X_train)
print ('mse_training = ', mean_squared_error(y_train, y_pred_train))
print ('r2_training = ', r2_score(y_train, y_pred_train))

y_pred_test = lin_reg.predict(X_test)
print ('mse_test = ', mean_squared_error(y_test, y_pred_test))
print ('r2_test = ', r2_score(y_test, y_pred_test))

R² ist mit etwa 0.56 bzw. 0.53 noch weit vom optimalen Wert 1 entfernt. Es besteht also noch Verbesserungspotential für die Regressionsfunktion!

### 4.3 Polynomiale Regression <a name="kap43"></a>

Betrachten Sie erneut das Streudiagramm zwischen `LSTAT` und `MEDV`: 

In [ ]:
train_set.plot(kind='scatter', x='LSTAT', y='MEDV');

Statt einer Geraden können auch andere Funktionen mit einer Regression angepasst werden. Betrachten Sie zunächst die Anpassung einer Parabel: 

Die Umsetzung in sklearn kann zunächst etwas verwirren: Der Befehl `poly_features.fit_transform()` erzeugt eine Menge, die neben den Werten von `LSTAT` nun in einer zweiten Spalte auch alle quadrierten Werte dieses Merkmals enthält. An diesen neuen Datensatz wird dann wieder das Modell `LinearRegression()` angepasst.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)  
X_polybig = poly_features.fit_transform(X_train)
print(X_polybig)  
q_reg = LinearRegression()
q_reg.fit(X_polybig, y_train)

In [ ]:
X_new_poly = poly_features.fit_transform(xline)
yline = q_reg.predict(X_new_poly)
plt.plot(X_train, y_train, '.')
plt.plot(xline,yline , color='red')
plt.xlabel('LSTAT')
plt.ylabel('MEDV')
plt.xlim(-0.1, 1.1)
plt.show()
y_pred = q_reg.predict(X_polybig)

print('Achsenabschnitt= ', q_reg.intercept_, 'Weitere_Koeffizienten= ', q_reg.coef_)
print ('mse= ', mean_squared_error(y_train, y_pred))
print ('r2= ', r2_score(y_train, y_pred))
X_polybig_test = poly_features.fit_transform(X_test)
y_pred_test = q_reg.predict(X_polybig_test)
print ('mse_test= ', mean_squared_error(y_test, y_pred_test))
print ('r2_test= ', r2_score(y_test, y_pred_test))

Das Polynom vom Grad 2 hat die Form:

$ \text{MEDV} = 0.7429774  -1.71341124 \text{ LSTAT} + 1.20149199 \text{ LSTAT}^2 $

Der mse wird kleiner, R² steigt auf 0.66 bzw. 0.60. Das Modell ist also besser geworden. Dies ist auch der visuelle Eindruck der Grafik. Dort ist auch zu sehen, dass ab ca. `LSTAT` = 0.7 die Regressionskurve wieder ansteigt. Dies scheint aber nicht der allgemeine Trend zu sein! 

Die Werte von mse und R² für die Testmenge zeigen wieder, dass das Modell sich schon stark den Trainingsdaten angepasst hat. 

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 

Auch der Grad des Polynoms kann noch weiter gesteigert werden. Berechnen und plotten Sie nun das Regressionspolynom für den Grad 3, also eine Funktion der Form $p (x) = a + bx + cx^2 + dx^3$. 
    
Beantworten Sie anschließend die folgenden Fragen: Ist die Regressionsfunktion nun monoton fallend im Bereich der Eingabewerte? Ist die Qualität des Modells noch weiter gestiegen?

</div>

In [ ]:
# Platz für Arbeitsauftrag

Es ergibt sich also:

$ \text{MEDV} = 0.80369577  -2.46637799 \text{ LSTAT} + 3.33601743 \text{ LSTAT}^2  -1.58977525 \text{ LSTAT}^3$

Es ist R² = 0.67 für die Trainingsdaten und 0.61 für die Testdaten, d.h. die Regressionspolynome zweiten und dritten Grades sind ungefähr gleich gut. Allerdings gibt es eine in den Kontext passende Verbesserung des Modells, da die Werte ab 0.7 nicht wieder ansteigen. 

### 4.4 Mehrdimensionale Regression <a name="kap44"></a>

Bisher wurde nur die Abhängigkeit von einem Eingabemerkmal zum Zielmerkmal betrachtet. Diese Vorgehensweise stößt schnell an ihre Grenzen, da das Zielmerkmal meistens von mehreren Merkmalen beeinflußt wird. 

Es wird nun versucht, das Modell dadurch zu verbessern, dass ein weiteres Merkmal, nämlich `RM` miteinbezogen wird. 

In [ ]:
X_train = train_set_scaled[['RM', 'LSTAT']] 
X_test = test_set_scaled[['RM', 'LSTAT']] 

m_reg = LinearRegression()
m_reg.fit(X_train, y_train)
print('Achsenabschnitt= ', m_reg.intercept_, 'Weitere_Koeffizienten= ', m_reg.coef_)

Es ergibt sich also:

$ \text{MEDV} = 0.23396104  +0.60512348 \text{ RM} -0.42334203 \text{ LSTAT}$

Nachfolgend wird das Modell visualisiert: 

In [ ]:
%matplotlib notebook

from mpl_toolkits.mplot3d import axes3d   

# Erstellen der Grafik
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Erstellen der Daten
Y1 = np.arange(0, 1.1, 0.1)
Y2 = np.arange(0, 1.1, 0.1)
Y1, Y2 = np.meshgrid(Y1, Y2)
Z = m_reg.intercept_  + m_reg.coef_[0][0]*Y1 + m_reg.coef_[0][1]*Y2

# Plot der Ebene
surf = ax.plot_surface(Y1, Y2, Z, color='red', linewidth=0, antialiased=False, alpha=0.2)

# Plot der Datenpunkte / Scatterplot
ax.scatter(X_train[['RM']], X_train[['LSTAT']], y_train)

# Beschriftung der Achsen
ax.set_xlabel('RM')
ax.set_ylabel('LSTAT')
ax.set_zlabel('MEDV')

plt.show()

Außerdem werden die üblichen Gütemaße bestimmt: 

In [ ]:
y_pred_train = m_reg.predict(X_train)   

print('achsenabschnitt= ', m_reg.intercept_, 'koeffizienten= ', m_reg.coef_)
print ('mse_train= ', mean_squared_error(y_train, y_pred_train))
print ('r2_train= ', r2_score(y_train, y_pred_train))

y_pred_test = m_reg.predict(X_test)
print ('mse_test= ', mean_squared_error(y_test, y_pred_test))
print ('r2_test= ', r2_score(y_test, y_pred_test))

Es ist R² = 0.68 für die Trainingsdaten und 0.61 für die Testdaten, d.h. die Regression entspricht von der Qualität etwa derjenigen mit den höhergradigen Polynomen. 

### 4.5 Lasso <a name="kap45"></a>

Bisher wurden die Merkmale zunächst über die Korrelationsmatrix ausgewählt und anschließend die Regression zu den ausgewählten Merkmalen bestimmt. Hierzu gibt es eine komfortable Alternative:

Bei der Lasso Regression wird nicht ein möglichst kleiner mse gewählt, um die Regressionsfunktion zu bestimmen, sondern der mse um einen zweiten Term erweitert: $mse + \alpha \cdot \sum_{i=1}^{m} |a_i|$.

Dieser zweite Term entspricht der Summe der Beträge der Koeffizienten, eine Minimierung des Gesamtausdruckes versucht also einen Kompromiss zu finden zwischen dem kleinsten mse bei gleichzeitig kleinen Koeffizienten, bzw. wenig Koeffizienten ungleich 0. Die Größenordnung des Parameters &alpha; sagt dabei aus, wie wichtig es ist, dass wenig bzw. kleine Koeffizienten ausgewählt werden. 

Für diese Art der Regression können also von vornherein alle Merkmale betrachtet werden: 

In [ ]:
X_train = train_set_scaled.drop(['MEDV'], axis = 1)
X_test = test_set_scaled.drop(['MEDV'], axis = 1)
X_train.head()

Für einen ersten Versuch wird der Parameter &alpha; nicht verändert, er befindet sich in der Standardvariante auf 1. Die Anwendung der Lasso Regression funktioniert wie alle bisherigen Modellierungen:  

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

print('Achsenabschnitt = ', lasso_reg.intercept_, 'Weitere_Koeffizienten = ', lasso_reg.coef_)

y_pred_train = lasso_reg.predict(X_train) 
print ('mse = ', mean_squared_error(y_train, y_pred_train))
print ('r2 = ', r2_score(y_train, y_pred_train))

Das Ergebnis ist ernüchternd: Die Regressionsfunktion lautet nun $ \text{MEDV} = 0.38524444$, alle weiteren Koeffizienten sind 0, und der mse ist stark gestiegen!

Im nächsten Ansatz wird der Paramter &alpha; variiert und die Ergebnisse miteinander verglichen: 

In [ ]:
for i in [0.001, 0.01, 0.1, 1]:
    print('alpha= ', i)
    lasso_reg = Lasso(alpha=i)
    lasso_reg.fit(X_train, y_train)

    print('Achsenabschnitt = ', lasso_reg.intercept_, 'Koeffizienten = ', lasso_reg.coef_)

    y_pred_train = lasso_reg.predict(X_train) 
    print ('mse = ', mean_squared_error(y_train, y_pred_train))
    print ('r2 = ', r2_score(y_train, y_pred_train))

    y_pred_test = lasso_reg.predict(X_test)
    print ('mse_test = ', mean_squared_error(y_test, y_pred_test))
    print ('r2_test = ', r2_score(y_test, y_pred_test))
    print('')

Die Ergebnisse zeigen den starken Einfluss von &alpha; auf das Modell: 

&alpha; = 0.001 ergibt das bisher beste Ergebnis, aber es werden alle Merkmale berücksichtigt, das Modell ist also recht aufwändig. 

Bei &alpha; = 0.01 gehen nur noch die Merkmale `INDUS`, `RM`, `TAX`, `PTRATIO` und `LSTAT` ins Modell ein. Die Gütemaße sind nun wieder schlechter, das Modell aber deutlich weniger komplex. 

Ab &alpha; = 0.1 werden, wie in der ersten Variante der Umsetzung, keine Merkmale mehr berücksichtigt.

## 5. Fazit <a name="kap5"></a>

Es wurden nun viele verschiedene Arten von Regression vorgestellt, hierbei wurde...

- für ein Eingabemerkmal eine Gerade, eine Parabel und ein Polynom dritten Grades angepasst.

- festgestellt, dass mehrdimensionale Modelle einen Trend in der Regel besser abbilden, aber auch aufwändiger sind.

- die Lasso Regression als Beispiel für andere Typen von Regressionsmodellen, die nicht nur das Ziel haben, den mse zu minimieren, vorgestellt. 